In [1]:
import ray
#ray.shutdown()
#ray.init()

In [2]:
from files import BCTrade_utils
from azureml.core import Workspace,Dataset

ws = Workspace.from_config()
name ='Bitstamp_2'
dataset = Dataset.get_by_name(ws, name=name)
env = BCTrade_utils.BCTrade(dataset.to_pandas_dataframe(),singlerun=False,spreadfactor=0.0)
experiment_name = BCTrade_utils.registerBCTrade(name)
from ray.tune.registry import register_env
def env_creator(env_config):
    return BCTrade_utils.BCTrade(dataset.to_pandas_dataframe())
register_env("BCTrade-Bitstamp_2", env_creator)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float64
  "Box bound precision lowered by casting to {}".format(self.dtype)


In [3]:
#datastore = ws.get_default_datastore()
#checkpoint_ds = Dataset.File.from_files(path=[(datastore,'checkpoints_BCTrade-Btc_with_indicators_1641664748_e3baadf3_head/checkpoint_370')])
#checkpoint = checkpoint_ds.as_named_input('test')
#checkpoint.as_mount('xxx')
#checkpoint = '/tmp/checkpoint_370/checkpoint-370'
#Users/rnikolaus/stuff/logs/PPO/PPO_BCTrade-Btc_with_indicators_5a87271e_2022-01-09_18-24-16nmvj7oep/checkpoint_100
checkpoint = '/home/azureuser/cloudfiles/code/Users/rnikolaus/stuff/'
checkpoint+='logs/PPO/PPO_BCTrade-Bitstamp_1_9dc72b58_2022-02-26_13-11-31dw8nnpvf/checkpoint_815/checkpoint-815'



#checkpoint = 'logs/PPO/PPO_BCTrade-Btc_with_indicators_5a87271e_2022-01-09_18-24-16nmvj7oep/checkpoint_110/checkpoint-110'

In [4]:
import ray
import ray.rllib.agents.ppo as ppo

#ray.init()
#config = {
  #"callbacks": {
  #  "on_train_result": "<function on_train_result at 0x7fca737a8e18>"
  #},
#  "env": "BCTrade-Btc_with_indicators",
#  "num_gpus": 0,
#  "num_workers": 1
#}

config = ppo.DEFAULT_CONFIG.copy()
config["num_gpus"] = 0
config["num_workers"] = 1
#config["eager"] = False
config["env"] = "BCTrade-Bitstamp_2"
new_trainer = ppo.PPOTrainer(config=config,env="BCTrade-Bitstamp_2")
new_trainer


2022-02-26 14:31:59,580	INFO services.py:1340 -- View the Ray dashboard at http://127.0.0.1:8265
(RolloutWorker pid=23849) /anaconda/envs/azureml_py36/lib/python3.6/site-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float64
(RolloutWorker pid=23849)   "Box bound precision lowered by casting to {}".format(self.dtype)
(RolloutWorker pid=23849) WARNING:tensorflow:From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1635: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
(RolloutWorker pid=23849) Instructions for updating:
(RolloutWorker pid=23849) If using Keras pass *_constraint arguments to layers.
(RolloutWorker pid=23849) 2022-02-26 14:32:24,898	WARNING deprecation.py:46 -- DeprecationWarning: `SampleBatch['is_training']` has been deprecated. Use `SampleBatch.is_training` 

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


PPO

In [5]:
import pickle
with open(checkpoint, "rb") as f:
        extra_data = pickle.load(f)

extra_data['worker']
weights=pickle.loads(extra_data['worker'])
weights['state']['default_policy']
new_trainer.get_policy().set_weights(weights['state']['default_policy'])
#new_trainer.get_policy().set_state(weights['state'])
#new_trainer.restore(checkpoint)
#ray.tune.run(ppo.PPOTrainer, config=config, restore='/home/azureuser/cloudfiles/code/Users/rnikolaus/stuff/logs/PPO/PPO_BCTrade-Btc_with_indicators_95fc9bdc_2022-01-10_21-38-0176w5xe80/checkpoint_822')

In [6]:
#obs = env.initialize()
obs = env.reset()
rewardsum=0
c=0
while True:
	action = new_trainer.compute_action(obs, explore=False)
	if obs[0]==1 and action ==2:
		env.render()
	obs, reward, done, _ = env.step(action)
	rewardsum+=reward
	c+=1
	if done:
		break
env.render()
print(rewardsum,rewardsum/c,c)


2022-02-26 14:32:58,220	WARNING deprecation.py:46 -- DeprecationWarning: `compute_action` has been deprecated. Use `compute_single_action` instead. This will raise an error in the future!


986.0755417690812
986.5381519207442
987.5379521603013
988.0327714197772
989.2637729924481
1012.484368442494
1015.0745781273837
1015.550845550776
1014.7536862035729
1013.1356294024375
1011.7410640268902
1012.1534935052197
1013.6960408566807
1013.1089565499601
1015.8319642636982
1017.3630566183676
1019.4385370120353
1020.304494747189
1021.8570749502491
1020.7059691321195
1027.8619748806761
1029.0649728918243
1015.789563624666
1015.0817924050062
1015.7854825990247
1014.6715721244873
1014.2176143837744
1013.9494005178722
1014.6234513661026
1013.7214298997346
1014.0152373725539
1013.112904291105
1010.8212156963373
1010.4934287242119
1010.7449389858707
1010.7449389858707
1011.8544785429528
1012.9608752129493
1012.6100508771397
1013.6809845388844
1014.7065881655838
1013.3210034841571
1013.7509739719511
1015.7052560635644
1015.8138538428719
1015.3829707891782
1014.8898323080805
1015.275804838499
1015.0418458631782
1014.3860640546832
1014.0611080018598
1014.4122390706585
1013.7420912076798
1014